# Modeling
### [1] 원핫 없이 통으로 학습 & 종목명 원핫 적용 후 학습 비교
### [2] 머신러닝 => 전체 돌리는 모듈로 상위 3개 정도 & 딥러닝 모델 비교

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [2]:
stock_df = pd.DataFrame()

files_m = os.listdir('../data/test_label/')

for min_df in tqdm(files_m[:1]):
    _df = pd.read_csv(f'../data/test_label/{min_df}', index_col='Unnamed: 0')
    stock_df = pd.concat([stock_df, _df])

100%|██████████| 1/1 [00:00<00:00,  2.54it/s]


In [3]:
# 첫 5줄을 봅니다.
stock_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,...,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률,label
20221103,1530,26450,26450,26450,26450,0,9183,242880000,56069,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,26450
20221103,1520,26700,26700,26600,26700,0,603,16090000,56069,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,26700
20221103,1519,26600,26700,26550,26700,0,803,21380000,55878,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,26700
20221103,1518,26600,26600,26550,26600,0,166,4410000,55848,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,26700
20221103,1517,26550,26600,26550,26600,0,1048,27870000,55814,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,26700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201027,905,33250,33300,33200,33300,0,2072,68900000,12609,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,34400
20201027,904,33200,33300,33150,33250,0,2362,78510000,12347,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,34400
20201027,903,33150,33300,33000,33200,0,2059,68200000,11944,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,34400
20201027,902,33150,33300,32800,33050,0,15054,496510000,10898,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,34400


In [4]:
def data_type_trans(stock_df):

    stock_df = stock_df.astype({'시간':'uint16'})
    stock_df = stock_df.astype({'시가':'uint32'})
    stock_df = stock_df.astype({'고가':'uint32'})
    stock_df = stock_df.astype({'저가':'uint32'})
    stock_df = stock_df.astype({'종가':'uint32'})
    stock_df = stock_df.astype({'전일대비':'int32'})
    stock_df = stock_df.astype({'거래량':'uint32'})
    stock_df = stock_df.astype({'거래대금':'uint64'})
    stock_df = stock_df.astype({'누적체결매도수량':'uint64'})
    stock_df = stock_df.astype({'상장주식수':'uint64'})
    stock_df = stock_df.astype({'시가총액':'uint64'})
    stock_df = stock_df.astype({'외국인주문한도수량':'uint64'})
    stock_df = stock_df.astype({'외국인주문가능수량':'uint64'})
    stock_df = stock_df.astype({'외국인현보유수량':'uint64'})
    stock_df = stock_df.astype({'외국인현보유비율':'float32'})
    stock_df = stock_df.astype({'수정주가일자':'uint32'})
    stock_df = stock_df.astype({'수정주가비율':'float32'})
    stock_df = stock_df.astype({'기관순매수량':'int64'})
    stock_df = stock_df.astype({'기관누적순매수량':'int64'})
    stock_df = stock_df.drop(['등락주선', '등락비율', '예탁금', '주식회전율', '거래성립률'], axis=1)
    stock_df = stock_df.astype({'label':'uint32'})

    stock_df.info()

    return stock_df

In [5]:
stock_df = data_type_trans(stock_df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190141 entries, 20221103 to 20201027
Data columns (total 20 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   시간         190141 non-null  uint16 
 1   시가         190141 non-null  uint32 
 2   고가         190141 non-null  uint32 
 3   저가         190141 non-null  uint32 
 4   종가         190141 non-null  uint32 
 5   전일대비       190141 non-null  int32  
 6   거래량        190141 non-null  uint32 
 7   거래대금       190141 non-null  uint64 
 8   누적체결매도수량   190141 non-null  uint64 
 9   상장주식수      190141 non-null  uint64 
 10  시가총액       190141 non-null  uint64 
 11  외국인주문한도수량  190141 non-null  uint64 
 12  외국인주문가능수량  190141 non-null  uint64 
 13  외국인현보유수량   190141 non-null  uint64 
 14  외국인현보유비율   190141 non-null  float32
 15  수정주가일자     190141 non-null  uint32 
 16  수정주가비율     190141 non-null  float32
 17  기관순매수량     190141 non-null  int64  
 18  기관누적순매수량   190141 non-null  int64  
 19  label      190

In [6]:
trans = stock_df.loc[:,['고가','label']]
trans = trans.rename(columns={'label':'pct_label'}).T

In [7]:
get_trans = trans.pct_change().T.iloc[:,-1]
get_trans

20221103    0.000000
20221103    0.000000
20221103    0.000000
20221103    0.003759
20221103    0.003759
              ...   
20201027    0.033033
20201027    0.033033
20201027    0.033033
20201027    0.033033
20201027    0.016248
Name: pct_label, Length: 190141, dtype: float64

In [8]:
stock_df = pd.concat([stock_df, get_trans], axis=1)
stock_df = stock_df.drop(['label'], axis=1)
stock_df



,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
20221103,1530,26450,26450,26450,26450,0,9183,242880000,56069,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1520,26700,26700,26600,26700,0,603,16090000,56069,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1519,26600,26700,26550,26700,0,803,21380000,55878,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1518,26600,26600,26550,26600,0,166,4410000,55848,0,0,0,0,0,0.0,0,0.0,0,0,0.003759
20221103,1517,26550,26600,26550,26600,0,1048,27870000,55814,0,0,0,0,0,0.0,0,0.0,0,0,0.003759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201027,905,33250,33300,33200,33300,0,2072,68900000,12609,0,0,0,0,0,0.0,0,0.0,0,0,0.033033
20201027,904,33200,33300,33150,33250,0,2362,78510000,12347,0,0,0,0,0,0.0,0,0.0,0,0,0.033033
20201027,903,33150,33300,33000,33200,0,2059,68200000,11944,0,0,0,0,0,0.0,0,0.0,0,0,0.033033
20201027,902,33150,33300,32800,33050,0,15054,496510000,10898,0,0,0,0,0,0.0,0,0.0,0,0,0.033033


In [9]:
stock_df['pct_label'] = stock_df['pct_label'].mul(100)
stock_df['pct_label'] = stock_df['pct_label'].round(0)
stock_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
20221103,1530,26450,26450,26450,26450,0,9183,242880000,56069,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1520,26700,26700,26600,26700,0,603,16090000,56069,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1519,26600,26700,26550,26700,0,803,21380000,55878,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1518,26600,26600,26550,26600,0,166,4410000,55848,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1517,26550,26600,26550,26600,0,1048,27870000,55814,0,0,0,0,0,0.0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201027,905,33250,33300,33200,33300,0,2072,68900000,12609,0,0,0,0,0,0.0,0,0.0,0,0,3.0
20201027,904,33200,33300,33150,33250,0,2362,78510000,12347,0,0,0,0,0,0.0,0,0.0,0,0,3.0
20201027,903,33150,33300,33000,33200,0,2059,68200000,11944,0,0,0,0,0,0.0,0,0.0,0,0,3.0
20201027,902,33150,33300,32800,33050,0,15054,496510000,10898,0,0,0,0,0,0.0,0,0.0,0,0,3.0


In [10]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
stock_df = stock_df.drop(columns=['상장주식수', '시가총액', '외국인주문한도수량', '외국인주문가능수량', '외국인현보유수량', '외국인현보유비율', '수정주가일자', '수정주가비율', '기관순매수량', '기관누적순매수량']
            , axis=1
)

X_stock_df = stock_df.iloc[:,0:-1]
y_stock_df = stock_df.iloc[:,-1]


In [11]:
from sklearn.model_selection import train_test_split

X_stock_sc = min_max_scaler.fit_transform(X_stock_df)

X_stock_trans_df = pd.DataFrame(data=X_stock_sc, columns=X_stock_df.columns)
X_stock_trans_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190141 entries, 0 to 190140
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시간        190141 non-null  float64
 1   시가        190141 non-null  float64
 2   고가        190141 non-null  float64
 3   저가        190141 non-null  float64
 4   종가        190141 non-null  float64
 5   전일대비      190141 non-null  float64
 6   거래량       190141 non-null  float64
 7   거래대금      190141 non-null  float64
 8   누적체결매도수량  190141 non-null  float64
dtypes: float64(9)
memory usage: 13.1 MB


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_stock_sc, y_stock_df, test_size=0.2, stratify=y_stock_df, random_state=42)

In [13]:
# X_stock_trans_df 데이터 변환 해줘야 함
X_stock_trans_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량
0,0.862826,0.147590,0.146269,0.156156,0.148036,0.0,0.059928,0.043025,0.047683
1,0.849108,0.162651,0.161194,0.165165,0.163142,0.0,0.003929,0.002847,0.047683
2,0.847737,0.156627,0.161194,0.162162,0.163142,0.0,0.005234,0.003784,0.047521
3,0.846365,0.156627,0.155224,0.162162,0.157100,0.0,0.001077,0.000778,0.047495
4,0.844993,0.153614,0.155224,0.162162,0.157100,0.0,0.006833,0.004934,0.047466
...,...,...,...,...,...,...,...,...,...
190136,0.005487,0.557229,0.555224,0.561562,0.561934,0.0,0.013517,0.012203,0.010723
190137,0.004115,0.554217,0.555224,0.558559,0.558912,0.0,0.015410,0.013905,0.010500
190138,0.002743,0.551205,0.555224,0.549550,0.555891,0.0,0.013432,0.012079,0.010158
190139,0.001372,0.551205,0.555224,0.537538,0.546828,0.0,0.098246,0.087958,0.009268


In [14]:
X_train

array([[0.27983539, 0.68072289, 0.67761194, ..., 0.00892851, 0.00856215,
        0.21900399],
       [0.69821674, 0.5813253 , 0.58208955, ..., 0.01281189, 0.01172977,
        0.17269186],
       [0.44855967, 0.63253012, 0.62985075, ..., 0.01130423, 0.01060304,
        0.2647846 ],
       ...,
       [0.00274348, 0.74096386, 0.74029851, ..., 0.00631131, 0.00622186,
        0.00145509],
       [0.55418381, 0.5813253 , 0.57910448, ..., 0.00317197, 0.00290011,
        0.05653946],
       [0.35253772, 0.63554217, 0.62985075, ..., 0.00225824, 0.00211884,
        0.0561253 ]])

In [15]:
# 레이블 카테고리 개수
y_stock_df.value_counts().count()

12

In [16]:
y_stock_df.value_counts()

0.0     125574
1.0      51989
2.0       8658
3.0       2207
4.0        685
8.0        256
5.0        238
6.0        183
10.0       156
9.0        125
7.0         60
11.0        10
Name: pct_label, dtype: int64

## 3. 원-핫 인코딩

In [80]:
# 원-핫 인코딩 처리를 합니다.
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

## 4. 소프트맥스

In [86]:
y_train

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
20210113,0,1,0,0,0,0,0,0,0,0,0,0
20210923,1,0,0,0,0,0,0,0,0,0,0,0
20210224,1,0,0,0,0,0,0,0,0,0,0,0
20211001,1,0,0,0,0,0,0,0,0,0,0,0
20210607,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
20220124,1,0,0,0,0,0,0,0,0,0,0,0
20211028,0,1,0,0,0,0,0,0,0,0,0,0
20220425,0,1,0,0,0,0,0,0,0,0,0,0
20201030,1,0,0,0,0,0,0,0,0,0,0,0


In [81]:
len(np.max(y_train))

c:\Users\TJ\.conda\envs\tf2\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


12

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# 모델 설정
model = Sequential()
model.add(Dense(1000,  input_dim=X_stock_df.shape[1], activation='relu'))
model.add(LSTM(X_stock_df.shape[1], activation='tanh'))
model.add(Dense(len(np.max(y_train)), activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X_train, y_train, epochs=30, batch_size=100, validation_data=(X_test, y_test))

model.evaluate(X_test, y_test)[1]

In [84]:
X_train.shape

(152112, 9)

In [17]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [18]:
y_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

X_train_lstm = X_train.reshape(X_train.shape[0], 9, 1)
X_test_lstm = X_test.reshape(X_test.shape[0], 9, 1)


In [20]:
y_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [21]:
print(y_train.shape)
print(X_train_lstm.shape)
print(X_train_lstm)

(152112, 12)
(152112, 9, 1)
[[[0.27983539]
  [0.68072289]
  [0.67761194]
  ...
  [0.00892851]
  [0.00856215]
  [0.21900399]]

 [[0.69821674]
  [0.5813253 ]
  [0.58208955]
  ...
  [0.01281189]
  [0.01172977]
  [0.17269186]]

 [[0.44855967]
  [0.63253012]
  [0.62985075]
  ...
  [0.01130423]
  [0.01060304]
  [0.2647846 ]]

 ...

 [[0.00274348]
  [0.74096386]
  [0.74029851]
  ...
  [0.00631131]
  [0.00622186]
  [0.00145509]]

 [[0.55418381]
  [0.5813253 ]
  [0.57910448]
  ...
  [0.00317197]
  [0.00290011]
  [0.05653946]]

 [[0.35253772]
  [0.63554217]
  [0.62985075]
  ...
  [0.00225824]
  [0.00211884]
  [0.0561253 ]]]


In [24]:

# 모델 설정
model = Sequential()
model.add(LSTM(10, input_shape=(9, 1), activation='tanh'))
model.add(Dense(12, activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X_train_lstm, y_train, epochs=10, batch_size=200,verbose=1)

# model.evaluate(X_test, y_test)[1]

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 10)                480       
                                                                 
 dense_2 (Dense)             (None, 12)                132       
                                                                 
Total params: 612
Trainable params: 612
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


InternalError: Graph execution error:

Detected at node 'sequential_2/dense_2/MatMul' defined at (most recent call last):
    File "c:\Users\TJ\.conda\envs\tf2\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\TJ\.conda\envs\tf2\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\TJ\.conda\envs\tf2\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\TJ\.conda\envs\tf2\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\TJ\.conda\envs\tf2\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\TJ\.conda\envs\tf2\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\TJ\.conda\envs\tf2\lib\asyncio\base_events.py", line 570, in run_forever
      